# Scripting with GeoIPS

This tutorial is intended to teach users how GeoIPS plugins can be used in
custom scripts.

Most current documentation for GeoIPS describes how to use the system
in a manner intended for near real-time processing and does not focus on
algorithm development and scripted use. However, it is possible, and sometimes
simpler, to use GeoIPS plugins in your own custom scripts. GeoIPS plugin
`interfaces` can be directly imported and used to load individual plugins. Those
plugins can then be called to manipulate your data.

## Plugins introduced

In this tutorial, we will use the following plugin types:
- reader
- sector
- interpolator
- output_formatter

## System requirements

- **CPU:** 1 CPU
- **RAM:** 40GB as the notebook is written, but could use more if modified.
  Reading the entire full-disk ABI image can take up to 100GB.
- **Disk Space:** 10GB storage space.

# Install ABI test data
Running the command below will use the GeoIPS CLI to download and install test data for ABI. The
download will consume about 2GB of disk space and will contain one complete
full-disk level 1-B dataset for both GOES-16 and GOES 18.

The data are installed by calling `geoips config install test_data_abi`. Other test datasets are
available. A list of available test datasets can be obtained by calling `geoips list test-datasets`.

📝 Note: If this command is run multiple times, it will not re-download the same dataset. If you need to update
a dataset, you will need to delete it, then re-download it.

In [ ]:
%%bash
echo "Installing ABI test data to ${GEOIPS_TESTDATA_DIR}"

# This command downloads the test_data_abi dataset
# This will be installed into $GEOIPS_TESTDATA_DIR
geoips config install test_data_abi

# Plugin Interfaces

GeoIPS provides programmatic access to its plugins through plugin `interfaces`.
Each plugin type (e.g. readers, algorithms, etc.) belongs to an interface.
Interfaces are used to load specific plugins and provide some limited ability to
discover plugins.

## List the available interfaces
While the CLI provides more useful interrogation functions, information about the GeoIPS artifacts can also be obtained programmatically. To retrieve a list of the available plugin interfaces (plugin types), call `geoips.interfaces.list_available_interfaces()`.

The cell below will print the available interfaces. There are several module-based and yaml-based interfaces in core GeoIPS, but no current text_based interfaces.

📝 Note: Additional details about each interface can be obtained via the CLI by calling `geoips describe interface <interface-name>`

In [ ]:
from pprint import pprint

# Import the interfaces module
from geoips import interfaces

# List the available interface
print("Available interfaces:")
print("---------------------")
pprint(interfaces.list_available_interfaces())

## Import the sectors interface and list the available sector plugins
Each interface has plugins registered to it. Those plugins are accessed via the interface object. To access sector plugins, use the `sectors` interface.

The cell below imports the sectors interface and prints a list of the available sector plugins.

In [ ]:
# Import the sectors interface, specifically
from geoips.interfaces import sectors

# List the available sectors
print("Available sectors:")
print("------------------")
pprint(sorted([sect.name for sect in sectors.get_plugins()]))

## Import the readers interface and list the available readers plugins
All interfaces behave the same. The cell below imports the readers interface and lists the available readers. Unlike the CLI, it does not provide additional context for the plugins.

📝 Note: At the time of writing this notebook, plugins do not have useful string representations so their names must be accessed via their `name` attribute. There is a pending issue to fix this. Once fixed, there would be no need for the loop used here.

In [ ]:
# Import the readers interface, specifically
from geoips.interfaces import readers

# List the available readers
print("Available readers:")
pprint(sorted([rdr.name for rdr in readers.get_plugins()]))

## Loading a Sector
Plugins can be loaded by calling the `get_plugin()` method of their interface. Below, we load the `conus` sector.

Each plugin can be inspected to better understand it. Yaml-based plugins like Sectors have a `yaml` attribute that prints the original yaml. `sector` plugins, additionally, have an `area_definition` attribute that contains the pyresample `AreaDefinition` for the sector

In [ ]:
# Load the CONUS sector and inspect it
conus = sectors.get_plugin("conus")

# Print the sector's details
print("Sector YAML definition:")
print("-----------------------")
pprint(conus.yaml)
print("\nArea Definition:")
print("----------------")
pprint(conus.area_definition)


In [ ]:
!geoips test sector conus

In [ ]:
from IPython.display import Image
import os

Image(f"{os.environ['GEOIPS_OUTDIRS']}/conus.png")

# Load the ABI L1B NetCDF reader
To load the reader plugin, again call `get_plugin()` on the interface object with the name of the plugin to load.

To get detailed information about the plugin in IPython, call `abi_reader?`. This method only works in IPython, but gives very useful information without needing to dig into the documentation. It will provide:
- Call Signature
- Plugin Type
- String Form of the Plugin
- Path to the plugin's file
- Docstring

In [ ]:
abi_reader = readers.get_plugin("abi_netcdf")
abi_reader?

## Reading ABI Data
Using the various plugins is straightforward. Using their particular call signature, simply call the plugin. To read ABI data, we construct a list of input files, then pass them to the ABI reader. The reader will read the files and output the results as a dictionary of xarrays where each key represents a data resolution.

In this case, we also specify two optional arguments:
- `area_def` tells the reader to use nearest neighbor interpolation to return data for a particular sector.
- `chans` tells the reader which channels to read.

Providing both of these options helps us limit the memory footprint of the data. Reading the full dataset requires about 100GB of RAM (which is expensive when serving this notebook in the cloud).

In [ ]:
import os
from glob import glob
import xarray as xr

# Collect the file list
files = glob(f"{os.environ['GEOIPS_TESTDATA_DIR']}/test_data_abi/data/goes16_20200918_1950/*")
print(f"Found {len(files)} files.")

# Read in ABI test data
# - files must be a list, not a glob pattern
# - We pass the area sector's area definition here to only read the data that we need.
#   This is significantly faster and uses much less memory (and costs less in cloud costs!)
#   Note that this will read the portion of the data that covers the area definition
#   but will keep all pixels in that region.
print("Reading ABI data - Channels 1, 2, and 3 reflectance")
# Describe what is returned here.
abi_reader = readers.get_plugin("abi_netcdf")
xarray_dict = abi_reader(files, area_def=conus.area_definition, chans=["B01Ref", "B02Ref", "B03Ref"])
print(xarray_dict.keys())
pprint(xarray_dict)


### Examining the data
The data returned are in a dictionary of Xarray DataSets. The names of the elements can be listed by calling `xarray_dict.keys()`.

In [ ]:
xarray_dict.keys()

One key is always present, `"METADATA"`. This key provides information about the dataset itself.

In [ ]:
xarray_dict["METADATA"]

All other elements have dynamic names. Each of these elements represents a specific shape of data. Here we only have `"conus"` because we specifically only read CONUS data. If, however, we had read the full ABI dataset without specifying a sector, the dictionary would contain DataSet elements for `"HIGH"`, `"MED"`, and `"LOW"`. These correspond to the three data resolutions provided by ABI.

To inspect the contents of one of these data arrays, simply print it.

In [ ]:
print(xarray_dict["conus"])

To access a particular variable, reference it by name like a dictionary element.

In [ ]:
print(xarray_dict["conus"]["B01Ref"])

You can also slice it like any normal array. For example, to get a 10x10 pixel slice:

In [ ]:
print(xarray_dict["conus"]["B01Ref"][600:610, 1200:1210])

Or to get every 4th pixel:

In [ ]:
print(xarray_dict["conus"]["B01Ref"][::4, ::4])

## Interpolating Data
Data can be interpolated to a new domain using interpolator plugins. While our data is already interpolated (to save on RAM) we can still re-apply the interpolation as an example.

Below, we import the interpolators interface, load the `interp_nearest` plugin, then interpolate each data resolution separately.

Note: Currently, each resolution needs to be interpolated separately, requiring a loop. Additionally, the `METADATA` dataset needs to be skipped. We have issues to address both of these issues in a backwards compatible way.

In [ ]:
from geoips.interfaces import interpolators

print("Loading interpolator plugin")
interpolator = interpolators.get_plugin("interp_nearest")

# Initialize an empty xarray instance into which the output will be placed
# This is a step that should disappear in the near future
interp_data = xr.Dataset()
for res in xarray_dict.keys():
    if res == "METADATA":
        continue
    print(f"Interpolating {res} resolution data")
    interp_data = interpolator(conus.area_definition, xarray_dict[res], interp_data, list(xarray_dict[res].data_vars))
pprint(interp_data)

### Plot the results
To see the results, you can just use Matplotlib like normal. Below we plot NIR, Red, and Blue as an RGB with no atmospheric correction. It isn't pretty both due to the lack of atmospheric correction and a lack of intention in which channels to combine.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib
from matplotlib import pyplot as plt

image = np.stack(
    [
        interp_data["B03Ref"].data,
        interp_data["B02Ref"].data,
        interp_data["B01Ref"].data,
    ],
    axis=-1,
)
print(image.shape)
print(matplotlib.get_backend())
plt.imshow(image)
plt.axis("off")
plt.show()

## Using an Algorithm
Algorithms can also be loaded and applied programmatically. Below we load the "single_channel" algorithm which, as it's name suggests, operates on a single variable.

Each plugin carries attributes that describe it. You can retrieve its `interface`, `family`, `name`, and other useful attributes. Running the cell below will load the "single_channel" algorithm and print these attributes.

In [ ]:
from geoips.interfaces import algorithms

# Load the single-channel algorithm
single_channel = algorithms.get_plugin("single_channel")
print(f"Interface: {single_channel.interface}")
print(f"Family: {single_channel.family}")
print(f"Name: {single_channel.name}")

The family of an algorithm is particularly important. It defines what its input and output data types are. In this case, the family is "list_numpy_to_numpy". This indicates that this algorithm expects a list of numpy arrays and returns a single numpy array.

***We are working to standardize all algorithms over the next year to use Xarray DataTrees but, until then, the families are vitally important.***

### Calling the Algorithm

To get an understanding of how to call the algorithm, we can get more information about it using the `?` trick again.

In [ ]:
single_channel?

`"single_channel"` is a fairly general algorithm, meant for use in many situations. As such, it has quite a few options. Let's apply just a few of them.
- We will pass `xarray_data["conus"]["B01Ref"]` as the input data (the blue band).
- We'll pass `output_data_range` to specify the range of the data we expect.
- `min_outbounds` and `max_outbouds` as "crop" to indicate to reduce any values outside our range to be either the min or max value.

What data range should we apply? Let's examine the data?

In [ ]:
print(xarray_dict["conus"]["B01Ref"].min(), xarray_dict["conus"]["B01Ref"].max())

Given the range, we can see that the data range from 0-1, kind of. Some values go beyond a reflectance of 1. How about we crop values at 1?

In [ ]:
%matplotlib inline

blue = single_channel([xarray_dict["conus"]["B01Ref"]], output_data_range=[0, 1], min_outbounds="crop", max_outbounds="crop")
plt.imshow(blue, cmap="Greys_r")
plt.axis("off")
plt.show()

# Last Thing
Let's make something more fun! Below is a full demonstration script that produces the Natural Color RGB.

In [ ]:
import os
from glob import glob
import xarray as xr
from geoips.interfaces import readers, sectors

# Collect the file list
files = glob(f"{os.environ['GEOIPS_TESTDATA_DIR']}/test_data_abi/data/goes16_20200918_1950/*")
print(f"Found {len(files)} files.")

sector = sectors.get_plugin("south_america")

# Read in ABI test data
# - files must be a list, not a glob pattern
# - We pass the area sector's area definition here to only read the data that we need.
#   This is significantly faster and uses much less memory (and costs less in cloud costs!)
#   Note that this will read the portion of the data that covers the area definition
#   but will keep all pixels in that region.

# Describe what is returned here.
print("Reading data - Channels 1, 2, 3")
abi_reader = readers.get_plugin("abi_netcdf")
xarray_dict = abi_reader(files, area_def=sector.area_definition, chans=["B02Ref", "B03Ref", "B05Ref"])
print("Done reading data")

# Build the channels
red = xarray_dict[sector.name]["B05Ref"]  # 1.6 um
grn = xarray_dict[sector.name]["B03Ref"]  # 0.86 um
blu = xarray_dict[sector.name]["B02Ref"]  # 0.64 um

img = np.dstack([red, grn, blu])

%matplotlib inline
# Should normally retrieve the correct DPI from matplotlib. Default is 100 DPI.
fig = plt.figure(figsize=(img.shape[1] / 100.0, img.shape[0] / 100.0))
ax = fig.add_axes((0, 0, 1, 1))
ax.imshow(img)
ax.axis("off")
plt.show()

# Play with it! Can you make it look better? Different sector? Different RGB recipe?

# Bonus! Why is this image dark? Can you use the data to figure out why?